## Load The Dependencies ##

In [1]:
import pandas as pd
import pdfplumber


## Load Text From PDF ##

In [2]:
def get_text(pdf_path):
    pdf_txt=""
    pdf = pdfplumber.open(pdf_path)
    for page in pdf.pages:
        pdf_txt += page.extract_text() 
    pdf.close()
    return pdf_txt

In [3]:
pdf = pdfplumber.open('../Documents/Amazon-Q4-2019-Earnings-Release.pdf')
page = pdf.pages[0]
page1 = pdf.pages[1]
pdf_txt = page.extract_text() + page1.extract_text()
pdf.close()

In [4]:
pdf_txt=get_text('../Documents/Amazon-Q4-2019-Earnings-Release.pdf')


In [5]:
pdf_txt

'AMAZON.COM ANNOUNCES FOURTH QUARTER SALES UP 21% TO $87.4 BILLION\nSEATTLE—(BUSINESS WIRE) January 30, 2020—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results\nfor its fourth quarter ended December 31, 2019.\nOperating cash flow increased 25% to $38.5 billion for the trailing twelve months, compared with $30.7 billion for the trailing\ntwelve months ended December 31, 2018. Free cash flow increased to $25.8 billion for the trailing twelve months, compared\nwith $19.4 billion for the trailing twelve months ended December 31, 2018. Free cash flow less principal repayments of finance\nleases and financing obligations increased to $16.2 billion for the trailing twelve months, compared with $11.6 billion for the\ntrailing twelve months ended December 31, 2018. Free cash flow less equipment finance leases and principal repayments of all\nother finance leases and financing obligations increased to $12.5 billion for the trailing twelve months, compared with $8.4\nbillion for th

In [6]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sagnik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
tokens = nltk.sent_tokenize(pdf_txt)
for t in tokens:
    print(t, "\n")

AMAZON.COM ANNOUNCES FOURTH QUARTER SALES UP 21% TO $87.4 BILLION
SEATTLE—(BUSINESS WIRE) January 30, 2020—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results
for its fourth quarter ended December 31, 2019. 

Operating cash flow increased 25% to $38.5 billion for the trailing twelve months, compared with $30.7 billion for the trailing
twelve months ended December 31, 2018. 

Free cash flow increased to $25.8 billion for the trailing twelve months, compared
with $19.4 billion for the trailing twelve months ended December 31, 2018. 

Free cash flow less principal repayments of finance
leases and financing obligations increased to $16.2 billion for the trailing twelve months, compared with $11.6 billion for the
trailing twelve months ended December 31, 2018. 

Free cash flow less equipment finance leases and principal repayments of all
other finance leases and financing obligations increased to $12.5 billion for the trailing twelve months, compared with $8.4
billion for the 

In [8]:
question="How many paid Prime members does Amazon have worldwide, according to Jeff Bezos' statement?"

In [11]:
pip install -U gensim


Note: you may need to restart the kernel to use updated packages.


In [9]:
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

def clean_sentence(sentence, stopwords=False):
  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
  if stopwords:
    sentence = remove_stopwords(sentence)
  return sentence

def get_cleaned_sentences(tokens, stopwords=False):
  cleaned_sentences = []
  for row in tokens:
    cleaned = clean_sentence(row, stopwords)
    cleaned_sentences.append(cleaned)
  return cleaned_sentences

In [10]:
cleaned_sentences = get_cleaned_sentences(tokens, stopwords=True)
cleaned_sentences_with_stopwords = get_cleaned_sentences(tokens, stopwords=False)
print(cleaned_sentences)
print(cleaned_sentences_with_stopwords)

['amazoncom announces fourth quarter sales 21 874 billion seattlebusiness wire january 30 2020amazoncom nasdaq amzn today announced financial results fourth quarter ended december 31 2019', 'operating cash flow increased 25 385 billion trailing months compared 307 billion trailing months ended december 31 2018', 'free cash flow increased 258 billion trailing months compared 194 billion trailing months ended december 31 2018', 'free cash flow principal repayments finance leases financing obligations increased 162 billion trailing months compared 116 billion trailing months ended december 31 2018', 'free cash flow equipment finance leases principal repayments finance leases financing obligations increased 125 billion trailing months compared 84 billion trailing months ended december 31 2018', 'common shares outstanding plus shares underlying stockbased awards totaled 512 million december 31 2019 compared 507 million year ago', 'fourth quarter 2019 net sales increased 21 874 billion fourt

**Question Answering with a Fine-Tuned BERT**

Bidirectional Encoder Representations from Transformers

Instead of looking at words in isolation, BERT, a transformer based model attempts to use the context of words to get embeddings. BERT uses several concepts of deep learning come up with a model that looks at context in a bidirectional fashion, leveraging information from the entire sentences as a whole through self-attention.

Example: Suppose Question is "How I can cross a road when their is no zebra crossing." and this question very similar to query "How I can cross road on zebra crossing." and thier is a chance that above two method give wrong answers but could handle that question. And we don't remove stoping words in the document because if it remove word like in above example "no" meaning of whole sentance become change. Bert uses stoping words to make relations between words.

*What is Model Fine-Tuning?*

BERT (Bidirectional Encoder Representations from Transformers) is a big neural network architecture, with a huge number of parameters, that can range from 100 million to over 300 million. So, training a BERT model from scratch on a small dataset would result in overfitting.

So, it is better to use a pre-trained BERT model that was trained on a huge dataset, as a starting point. We can then further train the model on our relatively smaller dataset and this process is known as model fine-tuning.

In [15]:
!pip install transformers


In [15]:
import torch
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [16]:
answer_text ='''Free cash flow less equipment finance leases and principal repayments of all
other finance leases and financing obligations increased to $12.5 billion for the trailing twelve months, compared with $8.4
billion for the trailing twelve months ended December 31, 2018. '''
question='What was the increase in free cash flow less equipment finance leases and principal repayments of all other finance leases and financing obligations for Amazon.com, Inc. for the trailing twelve months ended December 31, 2019, compared to the previous year?'

In [14]:
input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 101 tokens.


In [17]:
print(input_ids)

[101, 2054, 2001, 1996, 3623, 1999, 2489, 5356, 4834, 2625, 3941, 5446, 29597, 1998, 4054, 24565, 8163, 1997, 2035, 2060, 5446, 29597, 1998, 12135, 14422, 2005, 9733, 1012, 4012, 1010, 4297, 1012, 2005, 1996, 12542, 4376, 2706, 3092, 2285, 2861, 1010, 10476, 1010, 4102, 2000, 1996, 3025, 2095, 1029, 102, 2489, 5356, 4834, 2625, 3941, 5446, 29597, 1998, 4054, 24565, 8163, 1997, 2035, 2060, 5446, 29597, 1998, 12135, 14422, 3445, 2000, 1002, 2260, 1012, 1019, 4551, 2005, 1996, 12542, 4376, 2706, 1010, 4102, 2007, 1002, 1022, 1012, 1018, 4551, 2005, 1996, 12542, 4376, 2706, 3092, 2285, 2861, 1010, 2760, 1012, 102]


In [18]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    if id == tokenizer.sep_token_id:
        print('')
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')

[CLS]           101
what          2,054
was           2,001
the           1,996
increase      3,623
in            1,999
free          2,489
cash          5,356
flow          4,834
less          2,625
equipment     3,941
finance       5,446
leases       29,597
and           1,998
principal     4,054
repay        24,565
##ments       8,163
of            1,997
all           2,035
other         2,060
finance       5,446
leases       29,597
and           1,998
financing    12,135
obligations  14,422
for           2,005
amazon        9,733
.             1,012
com           4,012
,             1,010
inc           4,297
.             1,012
for           2,005
the           1,996
trailing     12,542
twelve        4,376
months        2,706
ended         3,092
december      2,285
31            2,861
,             1,010
2019         10,476
,             1,010
compared      4,102
to            2,000
the           1,996
previous      3,025
year          2,095
?             1,029

[SEP]           102

In [19]:
sep_index = input_ids.index(tokenizer.sep_token_id)
num_seg_a = sep_index + 1  # Include the [SEP] token in the first segment
num_seg_b = len(input_ids) - sep_index - 1
#Here We Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b
# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [20]:
print(sep_index)
print(num_seg_a)
print(num_seg_b)
print(segment_ids)

49
50
51
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [23]:
outputs = model(input_ids=torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))


In [24]:
print(outputs)
print(type(outputs))

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-5.7002, -5.4168, -7.3464, -6.7257, -5.8443, -8.2090, -7.4237, -8.1909,
         -8.8588, -9.0374, -8.3733, -8.8752, -8.6819, -8.2099, -6.8724, -7.9417,
         -8.7990, -8.0293, -8.3924, -8.9430, -7.7395, -8.2965, -8.5734, -6.0633,
         -8.1081, -7.6322, -3.3108, -7.7168, -7.3875, -8.2047, -6.8686, -9.2611,
         -6.6232, -6.2831, -6.1643, -3.8004, -6.9552, -5.5420, -3.8876, -5.2059,
         -7.3763, -1.8010, -7.8606, -6.0211, -5.9222, -3.5620, -3.7832, -3.7966,
         -8.6756, -5.7002,  3.1144, -0.4806, -3.2048, -2.6982, -0.7938, -2.3990,
         -1.2093, -4.0535, -0.2983, -1.9121, -3.9688, -3.6843, -0.6560, -3.5216,
         -3.5783, -3.1879, -5.4474, -2.4901, -2.7442,  3.8236,  1.2294,  7.0944,
          2.3764, -1.7887, -1.4384, -1.8555, -3.1560, -4.8591, -3.1658, -3.1338,
         -3.5299, -4.8023, -1.8669, -4.5807,  2.5840, -1.1312, -4.8190, -3.8132,
         -3.6473, -4.8831, -5.2807, -3.5299, -3.3416, -4

In [25]:
start_index = torch.argmax(outputs.start_logits)
end_index = torch.argmax(outputs.end_logits)

In [26]:
print(outputs.keys())


odict_keys(['start_logits', 'end_logits'])


In [27]:
answer = tokens[start_index]
#answer = tokens[end_index]

# Select the remaining answer tokens and join them with whitespace.
for i in range(start_index + 1, end_index + 1):
  # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
        # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]
print('Answer: "' + answer + '"')

Answer: "$ 12 . 5 billion"


In [28]:
def answer_question(question, answer_text):
    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
    print('Query has {:,} tokens.\n'.format(len(input_ids)))
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    outputs = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    score = float(torch.max(start_index))
    answer_start = torch.argmax(start_index)
    answer_end = torch.argmax(end_index)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer = tokens[start_index]
    for i in range(start_index + 1, end_index + 1):

        if tokens[i][0:2] == ' ':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]
    return answer, score

**Training the model on Random Samples**

In [29]:
answer_text = """The ability of the machine to infer knowledge from
the user documents can be tested based on its ability to answer
the question asked. Conventional Artificial Neural Network
(ANN) models for knowledge extraction only answer to the
questions which are simple and objective as they don't analyze
the questions and don't try to understand what really the content
of document mean. The proposed question answering system
(QAS) uses deep cases along with ANN to understand the
contents of the documents. We divide the sentences of natural
language into knowledge units and assign deep case to each word
to improve the quality of knowledge extraction."""

question = "How to improve the quality of knowledge extraction?"
ground_truth = "divide the sentences of natural language into knowledge units and assign deep case to each word"

question1 = "What is the capital of France?"
answer_text1 = "The capital of France is Paris."
ground_truth1 = "France"

question2 = "What is Artificial Intelligence?"
answer_text2 = "AI, or Artificial Intelligence, refers to the simulation of human intelligence in machines that are programmed to think and learn like humans."
ground_truth2 = "simulation of human intelligence"


In [31]:
answer_question(question1,answer_text1)


Query has 17 tokens.



('paris', 14.0)

In [32]:
answer_question(question2,answer_text2)


Query has 33 tokens.



('the simulation of human intelligence', 15.0)

## Modify function to calculate F1-score ##

In [34]:
from sklearn.metrics import f1_score


In [35]:
def answer_question_F1_Score(question, answer_text, ground_truth):
    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
    print('The input has a total of {:} tokens.'.format(len(input_ids)))
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    # Construct the list of 0s and 1s.
    segment_ids = [0] * num_seg_a + [1] * num_seg_b
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)
    outputs = model(input_ids=torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    predicted_answer = tokens[start_index]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(start_index + 1, end_index + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            predicted_answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            predicted_answer += ' ' + tokens[i]

    print('Predicted Answer: "' + predicted_answer + '"')
    print('Ground Truth Answer: "' + ground_truth + '"')
    # Calculate F1 score
    y_true = [0] * len(tokens)  # 0 for tokens outside the answer
    y_true[start_index:end_index + 1] = [1] * (end_index - start_index + 1)  # 1 for tokens within the answer
    y_pred = [1 if i >= start_index and i <= end_index else 0 for i in range(len(tokens))]
    f1 = f1_score(y_true, y_pred)
    print('F1 Score:', f1)

In [36]:
answer_question_F1_Score(question2, answer_text2, ground_truth2)


The input has a total of 33 tokens.
Predicted Answer: "the simulation of human intelligence"
Ground Truth Answer: "simulation of human intelligence"
F1 Score: 1.0


**Limitation of Bert that it process only upto 512 tokens So we can implement new method to make it for whole document.**


In [37]:

pdf_txt

'AMAZON.COM ANNOUNCES FOURTH QUARTER SALES UP 21% TO $87.4 BILLION\nSEATTLE—(BUSINESS WIRE) January 30, 2020—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results\nfor its fourth quarter ended December 31, 2019.\nOperating cash flow increased 25% to $38.5 billion for the trailing twelve months, compared with $30.7 billion for the trailing\ntwelve months ended December 31, 2018. Free cash flow increased to $25.8 billion for the trailing twelve months, compared\nwith $19.4 billion for the trailing twelve months ended December 31, 2018. Free cash flow less principal repayments of finance\nleases and financing obligations increased to $16.2 billion for the trailing twelve months, compared with $11.6 billion for the\ntrailing twelve months ended December 31, 2018. Free cash flow less equipment finance leases and principal repayments of all\nother finance leases and financing obligations increased to $12.5 billion for the trailing twelve months, compared with $8.4\nbillion for th

In [38]:
user_question1 = "How aws helped amazon to grow in the year 2022?"


We need to split the sentences according to tokens i.e, we can split upto 512 tokens after that we need to add some special token like [SEP], [CLS].

Tokenization: Tokenization is the process of converting a string of text into a list of tokens (individual words/punctuation) and/or token IDs (integers that map a word to a vector representation of that word in an embedding array).

Here we are using the tokenizers encode_plus method to create our tokens from the text string add_special_tokens=True adds special BERT tokens like [CLS], [SEP], and [PAD] to our new ‘tokenized’ encodings max_length=512 tells the encoder the target length of our encodings truncation=True ensures we cut any sequences that are longer than the specified max_length . padding="max_length" tells the encoder to pad any sequences that are shorter than the max_length with padding tokens.

In [39]:
tokens = tokenizer.encode_plus(user_question1, pdf_txt, add_special_tokens=True, max_length=512, truncation=True, padding="max_length")
tokens

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'input_ids': [101, 2129, 22091, 2015, 3271, 9733, 2000, 4982, 1999, 1996, 2095, 16798, 2475, 1029, 102, 9733, 1012, 4012, 17472, 2959, 4284, 4341, 2039, 2538, 1003, 2000, 1002, 6584, 1012, 1018, 4551, 5862, 1517, 1006, 2449, 7318, 1007, 2254, 2382, 1010, 12609, 1517, 9733, 1012, 4012, 1010, 4297, 1012, 1006, 17235, 2850, 4160, 1024, 2572, 2480, 2078, 1007, 2651, 2623, 3361, 3463, 2005, 2049, 2959, 4284, 3092, 2285, 2861, 1010, 10476, 1012, 4082, 5356, 4834, 3445, 2423, 1003, 2000, 1002, 4229, 1012, 1019, 4551, 2005, 1996, 12542, 4376, 2706, 1010, 4102, 2007, 1002, 2382, 1012, 1021, 4551, 2005, 1996, 12542, 4376, 2706, 3092, 2285, 2861, 1010, 2760, 1012, 2489, 5356, 4834, 3445, 2000, 1002, 2423, 1012, 1022, 4551, 2005, 1996, 12542, 4376, 2706, 1010, 4102, 2007, 1002, 2539, 1012, 1018, 4551, 2005, 1996, 12542, 4376, 2706, 3092, 2285, 2861, 1010, 2760, 1012, 2489, 5356, 4834, 2625, 4054, 24565, 8163, 1997, 5446, 29597, 1998, 12135, 14422, 3445, 2000, 1002, 2385, 1012, 1016, 4551, 2005, 1

It return a dictionary containing three key-value pairs, input_ids , token_type_ids , and attention_mask . We have also added return_tensors='pt' to return PyTorch tensors from the tokenizer (rather than Python lists).

In [40]:
tokens = tokenizer.encode_plus(pdf_txt, add_special_tokens=False, return_tensors='pt')
tokens

Token indices sequence length is longer than the specified maximum sequence length for this model (12108 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[ 9733,  1012,  4012,  ...,  4012,  1013, 10975]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [41]:
input_id_chunks = tokens['input_ids'][0].split(510)
mask_chunks = tokens['attention_mask'][0].split(510)

for tensor in input_id_chunks:
  print(len(tensor))

510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
510
378


In [42]:
a = torch.arange(10)
a

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [43]:
torch.cat(
    [torch.Tensor([101]), a, torch.Tensor([102])]
)

tensor([101.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9., 102.])

Preparing The Chunks

Now we have our tokenized tensor; we need to break it into chunks of no more than 510 tokens. We choose 510 rather than 512 to leave two places spare to add our [CLS] and [SEP] tokens.

Split

We apply the split method to both our input IDs and attention mask tensors (we don’t need the token type IDs and can discard them). Now we have three chunks for each tensor set. Note that we will need to add padding to the final chunk as it will not satisfy the tensor size of 512 required by BERT.

CLS and SEP

Next, we add the start of sequence [CLS] and separator [SEP] tokens. For this, we can use the torch.cat function, which concatenates a list of tensors. Our tokens are already in token ID format, so we can refer to the special tokens table above to create the token ID versions of our [CLS] and [SEP] tokens. Because we are doing this for multiple tensors, we place the torch.cat function into a for-loop and perform the concatenation for each of our chunks individually. Additionally, our attention mask chunks are concatenated with 1s rather than 101 and 102. We do this because the attention mask does not contain token IDs but instead a set of 1s and 0s. Zeros in the attention mask represent the location of padding tokens (which we will add next), and as [CLS] and [SEP] are not padding tokens, they are represented with 1s.

Padding

We need to add padding to our tensor chunks to ensure they satisfy the 512 tensorlength required by BERT. Our first two chunks don’t require any padding as they already satisfy this length requirement, but the final chunks do. To check if a chunk requires padding, we add an if-statement that checks the tensor length. If the tensor is shorter than 512 tokens, we add padding using the torch.cat function. We should add this statement to the same for-loop where we add our [CLS] and [SEP] tokens

In [44]:
chunksize = 512

input_id_chunks = list(tokens['input_ids'][0].split(chunksize - 2))
mask_chunks = list(tokens['attention_mask'][0].split(chunksize - 2))

for i in range(len(input_id_chunks)):
    input_id_chunks[i] = torch.cat([
        torch.tensor([101]), input_id_chunks[i], torch.tensor([102])
    ])
    mask_chunks[i] = torch.cat([
        torch.tensor([1]), mask_chunks[i], torch.tensor([1])
    ])

    pad_len = chunksize - input_id_chunks[i].shape[0]
    if pad_len > 0:
        input_id_chunks[i] = torch.cat([
            input_id_chunks[i], torch.Tensor([0] * pad_len)
        ])
        mask_chunks[i] = torch.cat([
            mask_chunks[i], torch.Tensor([0] * pad_len)
        ])

for chunk in input_id_chunks:
    print(chunk)

tensor([  101,  9733,  1012,  4012, 17472,  2959,  4284,  4341,  2039,  2538,
         1003,  2000,  1002,  6584,  1012,  1018,  4551,  5862,  1517,  1006,
         2449,  7318,  1007,  2254,  2382,  1010, 12609,  1517,  9733,  1012,
         4012,  1010,  4297,  1012,  1006, 17235,  2850,  4160,  1024,  2572,
         2480,  2078,  1007,  2651,  2623,  3361,  3463,  2005,  2049,  2959,
         4284,  3092,  2285,  2861,  1010, 10476,  1012,  4082,  5356,  4834,
         3445,  2423,  1003,  2000,  1002,  4229,  1012,  1019,  4551,  2005,
         1996, 12542,  4376,  2706,  1010,  4102,  2007,  1002,  2382,  1012,
         1021,  4551,  2005,  1996, 12542,  4376,  2706,  3092,  2285,  2861,
         1010,  2760,  1012,  2489,  5356,  4834,  3445,  2000,  1002,  2423,
         1012,  1022,  4551,  2005,  1996, 12542,  4376,  2706,  1010,  4102,
         2007,  1002,  2539,  1012,  1018,  4551,  2005,  1996, 12542,  4376,
         2706,  3092,  2285,  2861,  1010,  2760,  1012,  2489, 

In [45]:
for tensor in range(len(input_id_chunks)):
  ans = answer_question(user_question1, ' '.join(tokenizer.convert_ids_to_tokens(input_id_chunks[tensor])))
  print(ans)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Query has 512 tokens.

('[SEP]', 511.0)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Query has 512 tokens.



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

('[CLS]', 0.0)
Query has 512 tokens.



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


('[SEP]', 14.0)
Query has 512 tokens.



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


('[SEP]', 511.0)
Query has 512 tokens.

('[CLS] how aw ##s helped amazon to grow in the year 202 ##2 ? [SEP] [CLS] ) as a result of revenue recognition accounting guidance adopted on january 1 , 2018 , certain advertising services are classified as revenue rather than a reduction in cost of sales . ( 7 ) exclude # # s the impact of whole foods market . amazon . com , inc . certain definitions customer accounts • references to customers mean customer accounts established when a customer places an order through one of our stores . customer accounts exclude certain customers , including customers associated with certain of our acquisitions , amazon payments customers , aw # # s customers , and the customers of select companies with whom we have a technology alliance or marketing and promotional relationship . customers are considered active when they have placed an order during the preceding twelve - month period . seller accounts • references to sellers means seller accounts , which are 

In [47]:
def expand_split_sentences(pdf_txt):
  nltk.download('punkt')
  new_chunks = nltk.sent_tokenize(pdf_txt)
  length = len(new_chunks)
  new_df = []
  for i in range(length):
    paragraph = ""
    for j in range(i, length):
      #tmp_str = paragraph + new_chunks[j]
      tmp_token = tokenizer.encode(paragraph + new_chunks[j])
      length_token = len(tmp_token)
      if length_token < 510:
        #print(length_token)
        paragraph = paragraph + new_chunks[j]
      else:
        #print(length_token)
        break;
    #print(len(tokenizer.encode(paragraph)))
    new_df.append(paragraph)
  return new_df

In [48]:
user_question2 = "What were Amazon's net sales in the first quarter of 2023?"
ground_truth ="Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in the first quarter 2022."
pdf_text = "SEATTLE—(BUSINESS WIRE) April 27, 2023—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results for its first quarter ended March 31, 2023. • Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in first quarter 2022. Excluding the $2.4 billion unfavorable impact from year-over-year changes in foreign exchange rates throughout the . . . The company debuted additional Amazon Original films and series, including action comedy Shotgun Wedding, starring Jennifer Lopez and Josh Duhamel; romantic comedy Somebody I Used to Know, directed by Dave Franco; comedic thriller The Consultant, starring Christoph Waltz; and Swarm, from co-creator and executive producer Donald Glover."
new_df = expand_split_sentences(pdf_text)
for new_context in new_df:
  #new_paragrapgh = new_paragrapgh + answer_question(question, answer_text)
  answer_question_F1_Score(user_question2, new_context,ground_truth)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sagnik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The input has a total of 178 tokens.
Predicted Answer: "$ 127 . 4 billion"
Ground Truth Answer: "Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in the first quarter 2022."
F1 Score: 1.0
The input has a total of 136 tokens.
Predicted Answer: "$ 127 . 4 billion"
Ground Truth Answer: "Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in the first quarter 2022."
F1 Score: 1.0
The input has a total of 106 tokens.
Predicted Answer: "$ 2 . 4 billion"
Ground Truth Answer: "Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in the first quarter 2022."
F1 Score: 1.0
The input has a total of 80 tokens.
Predicted Answer: "what were amazon ' s net sales in the first quarter of 2023 ? [SEP] . . the company debuted additional amazon original films"
Ground Truth Answer: "Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in the first q

In [49]:
def answer_question3(question, answer_text, ground_truth):
    # Tokenize and preprocess the ground truth
    ground_truth_processed = re.sub(r' ##', '', ground_truth)
    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
    print('The input has a total of {:} tokens.'.format(len(input_ids)))
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    # Construct the list of 0s and 1s.
    segment_ids = [0] * num_seg_a + [1] * num_seg_b
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)
    outputs = model(input_ids=torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    predicted_answer = tokens[start_index]
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(start_index + 1, end_index + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            predicted_answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            predicted_answer += ' ' + tokens[i]
    # Tokenize and preprocess the predicted answer
    predicted_answer_processed = re.sub(r' ##', '', predicted_answer)
    # Ensure that ground truth and predicted answer have the same length
    ground_truth_processed = ground_truth_processed[:len(predicted_answer_processed)]
    # Calculate F1 score for ground truth and predicted answer
    f1_ground_truth_predicted = f1_score(list(ground_truth_processed), list(predicted_answer_processed), average='weighted')
    return predicted_answer, f1_ground_truth_predicted

In [51]:
df = pd.read_csv("../Documents/Curated datasets created from PDFs/question_answer_dataset_financial.csv")


In [53]:
df.head()

,Question,Context,GroundTruth
0,What were Amazon's net sales in the first quar...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",Net sales increased 9% to $127.4 billion in th...
1,How much did net sales increase compared to th...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",Excluding the $2.4 billion unfavorable impact ...
2,What was the impact of foreign exchange rates ...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",Excluding the $2.4 billion unfavorable impact ...
3,How did North America segment sales change yea...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",North America segment sales increased 11% year...
4,What was the percentage increase in AWS segmen...,"\nSEATTLE—(BUSINESS WIRE) April 27, 2023—Amazo...",AWS segment sales increased 16% year-over-year...


In [52]:
# Loop through rows and call the function
for index, row in df.iterrows():
    user_question = row['Question']
    context = row['Context']
    ground_truth = row['GroundTruth']
    # Call your answer_question function
    ans, f1 = answer_question3(user_question,context,ground_truth)
    # Print the result
    print(f"Question: {user_question}")
    print(f"Context: {context}")
    print(f"Answer: {ans}")
    print(f"F1 Score: {f1}")
    print("=" * 50)

The input has a total of 178 tokens.
Question: What were Amazon's net sales in the first quarter of 2023?
Context: 
SEATTLE—(BUSINESS WIRE) April 27, 2023—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results
for its first quarter ended March 31, 2023.
• Net sales increased 9% to $127.4 billion in the first quarter, compared with $116.4 billion in first quarter 2022.
Excluding the $2.4 billion unfavorable impact from year-over-year changes in foreign exchange rates throughout the
 .
 .
 .
The company debuted additional Amazon Original films and series, including action comedy Shotgun
Wedding, starring Jennifer Lopez and Josh Duhamel; romantic comedy Somebody I Used to Know, directed by Dave
Franco; comedic thriller The Consultant, starring Christoph Waltz; and Swarm, from co-creator and executive
producer Donald Glover.

Answer: $ 127 . 4 billion
F1 Score: 0.0392156862745098
The input has a total of 178 tokens.
Question: How much did net sales increase compared to the first